In [5]:
import torch
import torch.nn as nn
from torchvision import models  # Import the models module

# Load the saved model
model_save_path = "vgg16_model.pth"  # Replace with your actual model path
model_state = torch.load(model_save_path, map_location=torch.device('cpu'), weights_only=True)  # Use 'cuda' if using GPU

# Extract the stored values
state_dict = model_state['state_dict']
nutrition_data = model_state['nutrition_data']
class_names = model_state['class_names']

# Reinitialize the VGG-16 model
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(class_names))

# Load the trained weights
model.load_state_dict(state_dict)
model.eval()  # Set the model to evaluation mode

print("VGG-16 model loaded successfully!")


E:\AISC LABS\ANACONDA\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\AISC LABS\ANACONDA\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG-16 model loaded successfully!


In [6]:
# Define a function to clean the class label
def clean_class_label(class_label):
    """
    Extract the fruit or vegetable name from the class label.
    E.g., 'Carrot__Rotten' -> 'Carrot'
    """
    return class_label.split('__')[0].strip()

# Define a function to predict the class and display nutrition info
def predict_and_display_vgg16(image_path, model, transform, class_names, device, nutrition_data):
    """
    Predict the class of an image using VGG-16 and display nutrition information if applicable.
    """
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)  

    # Set model to evaluation mode and make prediction
    model.eval()
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output, 1)

    # Get the predicted class label
    predicted_class = class_names[predicted.item()]
    fruit_or_veg_name = clean_class_label(predicted_class)

    # Determine state (Healthy or Rotten)
    is_healthy = "Healthy" if "Healthy" in predicted_class else "Rotten"
    print(f"State: {is_healthy}")
    print("Fruit or Vegetable Name: " + fruit_or_veg_name)

    # Skip nutrition retrieval if the fruit/vegetable is Rotten
    if is_healthy == "Rotten":
        print("This is rotten and hence not safe to consume.")
        return

    # Get nutrition values for the predicted fruit or vegetable
    nutrition = pd.DataFrame(nutrition_data)
    nutrition = nutrition[nutrition['name'].str.contains(fruit_or_veg_name, case=False, na=False)]
    if not nutrition.empty:
        nutrition = nutrition.iloc[0]
        nutrition_data = {
            'Nutrient': ['Energy (kJ)', 'Water (g)', 'Protein (g)', 'Total Fat (g)', 'Carbohydrates (g)', 'Fiber (g)', 
                         'Sugars (g)', 'Calcium (mg)', 'Iron (mg)', 'Magnesium (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 
                         'Sodium (g)', 'Vitamin A (IU)', 'Vitamin C (mg)', 'Vitamin B1 (mg)', 'Vitamin B2 (mg)', 
                         'Vitamin B3 (mg)', 'Vitamin B5 (mg)', 'Vitamin B6 (mg)', 'Vitamin E (mg)'],
            'Value': [nutrition['energy (kJ)'], nutrition['water (g)'], nutrition['protein (g)'], nutrition['total fat (g)'], 
                      nutrition['carbohydrates (g)'], nutrition['fiber (g)'], nutrition['sugars (g)'], nutrition['calcium (mg)'], 
                      nutrition['iron (mg)'], nutrition['magnessium (mg)'], nutrition['phosphorus (mg)'], nutrition['potassium (mg)'], 
                      nutrition['sodium (g)'], nutrition['vitamin A (IU)'], nutrition['vitamin C (mg)'], nutrition['vitamin B1 (mg)'], 
                      nutrition['vitamin B2 (mg)'], nutrition['vitamin B3 (mg)'], nutrition['vitamin B5 (mg)'], 
                      nutrition['vitamin B6 (mg)'], nutrition['vitamin E (mg)']]
        }

        # Convert to DataFrame for better visualization
        nutrition_df_table = pd.DataFrame(nutrition_data)

        # Display the table
        print("\nNutrition Information:")
        print(nutrition_df_table.to_string(index=False))
    else:
        print("No matching nutrition data found for this fruit or vegetable.")


In [8]:
from torchvision import transforms  # Import transforms module from torchvision

# Define the transform (same as used during training)
transformer = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])


In [9]:
# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [13]:
# Example usage
from PIL import Image  # Import the Image module from PIL
import pandas as pd

image_path = (r"D:\FRUITSANDVEGETABLESDATASET\Fruit And Vegetable Diseases Dataset\Cucumber__Healthy\freshCucumber (1).jpeg")  # Replace with your image path
predict_and_display_vgg16(image_path, model, transformer, class_names, device, nutrition_data)

State: Healthy
Fruit or Vegetable Name: Cucumber

Nutrition Information:
         Nutrient  Value
      Energy (kJ)  62.76
        Water (g)   95.2
      Protein (g)   0.65
    Total Fat (g)   0.11
Carbohydrates (g)   3.63
        Fiber (g)    0.5
       Sugars (g)   1.67
     Calcium (mg)     16
        Iron (mg)   0.28
   Magnesium (mg)     13
  Phosphorus (mg)     24
   Potassium (mg)    147
       Sodium (g)      2
   Vitamin A (IU)    105
   Vitamin C (mg)    2.8
  Vitamin B1 (mg)  0.027
  Vitamin B2 (mg)  0.033
  Vitamin B3 (mg)  0.098
  Vitamin B5 (mg)  0.259
  Vitamin B6 (mg)   0.04
   Vitamin E (mg)   0.03
